In [38]:
from spacy import Language
import glob
import pprint
import re
import os

import lxml.etree
from lxml import etree
from lxml.etree import XMLSyntaxError
import pandas as pd
import numpy as np
import spacy
from spacy_langdetect import LanguageDetector

@Language.factory('language_detector')
def language_detector(nlp, name):
    return LanguageDetector()

NLP = spacy.load('de_core_news_sm')
NLP.add_pipe('language_detector')

In [45]:
NAMESPACE: dict = {'tei': 'http://www.tei-c.org/ns/1.0'}
SAVINGS_PATH: str = '../data/plain-text/'

if not os.path.exists(SAVINGS_PATH):
    os.makedirs(SAVINGS_PATH)

In [49]:
def get_letter_text(path='../data/*.xml'):

    for file in glob.glob(pathname=path)[:200]:
        try:
            tree = etree.parse(file)

            letter_output: str = ""

            for text in tree.xpath('//tei:text//tei:body//text()', namespaces=NAMESPACE):
                text = text.lower()
                if re.match(r'.*\[.*].*', text):
                    letter_output += re.sub(r'\[.*]', '', text) + ' '
                elif 'beyliegendes von' or 'paginierung des editors' in text:
                    letter_output += re.sub(r'beyliegendes von|paginierung des editors', '', text) + ' '
                else:
                    letter_output += re.sub(r'^(0[1-9]|[12][0-9]|3[01])[.](0[1-9]|1[012])[.](19|20)[0-9]{2}', ' ', text) + ' '

            letter_output = re.sub('\s{2,}', ' ', letter_output.lstrip())
            letter_output = re.sub('[^\w\s\d]|_', '', letter_output)


            doc = NLP(letter_output)
            if doc._.language['language'] == 'de':
                with open(SAVINGS_PATH + re.sub(r'\.xml', '.txt', os.path.basename(file)), 'w') as f_tbw:
                    f_tbw.write(letter_output)

        except XMLSyntaxError:
            pass



In [50]:
get_letter_text()

In [58]:
from textblob_de import TextBlobDE

In [82]:
sentiment_dict: dict = {}

for file in glob.glob('../data/plain-text/*.txt')[:]:
    with open(file, 'r') as f:
        text = f.read()
        letter_blob = TextBlobDE(text)
        sentiment_dict[os.path.basename(file)] = letter_blob.sentiment

s_df = pd.DataFrame.from_dict(sentiment_dict).transpose()
s_df.index.names = ['filename']
s_df.rename(columns={0: 'polarity', 1: 'subjectivity'}, inplace=True)
s_df.to_csv('../data/retrieved/sentiments.csv')
